In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import pandas as pd
import numpy as np
import os
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras import optimizers
import matplotlib.pyplot as plt
#import argparse
#import random
#import pickle
#import cv2
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

## 데이터 불러오기

In [ ]:
x_train = pd.read_csv('X_data.csv',engine='python',encoding = "euc-kr")
y_train = pd.read_csv('Y_data_cpu_gpu.csv',engine='python',encoding = "euc-kr")
recom_reference = pd.read_csv('제품사양.csv',engine='python',encoding = "euc-kr")
predic_X = pd.read_csv('X_test.csv',engine='python',encoding = "euc-kr")

In [ ]:
y_train[y_train >=0.5] = 1
y_train[y_train < 0.5] = 0

In [ ]:
X_train, X_test , Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.1, random_state = 5)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 7)

In [ ]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(units=64, kernel_initializer='uniform', input_dim=47, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=32, kernel_initializer='uniform', activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=32, kernel_initializer='uniform', activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(166, activation='softmax')
  ])

  optimizer = tf.keras.optimizers.Adam(0.01)
  #optimizer = tf.keras.optimizers.RMSprop(0.01)
  
  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
  return model
model = build_model()
model.summary()

In [ ]:
EPOCHS=1000

In [ ]:
history = model.fit(X_train,Y_train,validation_data=(X_val,Y_val),
          epochs=EPOCHS,batch_size=64,verbose=2)

In [ ]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

# 유사도 기반 추천

In [ ]:
predic_Y = model.predict(predic_X)[0]

In [ ]:
name = recom_reference['제품']
del recom_reference['제품']
recom_reference = recom_reference.as_matrix()

In [ ]:
dist_mat = np.zeros((1,2195))[0]

In [ ]:
len_r = recom_reference.shape[0]
for a in range(len_r):
  dist_mat[a] = sum((predic_Y - recom_reference[a])**2)

In [ ]:
pd.concat([pd.DataFrame({'score' : dist_mat}),name],axis=1).sort_values(by=['score'],axis=0).head()

In [ ]:
from keras.models import load_model

In [ ]:
model.save('model5.h5')